In [1]:
import argparse
import csv
import json
import sys
from typing import Any, Dict, List, Tuple

# Define constants that are used but not defined in the snippet
DEFAULT_ENDPOINT = "https://overpass-api.de/api/interpreter"
USER_AGENT = "YourAppName/1.0 (you@example.com)"
OVERPASS_QUERY = """
[out:json];
area["name"="Berlin"]["admin_level"="4"]->.berlin;
(
  node["healthcare"="dentist"](area.berlin);
  way["healthcare"="dentist"](area.berlin);
  relation["healthcare"="dentist"](area.berlin);
);
out center;
"""

# Function to normalize elements (placeholder implementation)
def normalize_element(e):
    # Implement according to your needs
    return e

# Function to convert elements to GeoJSON (placeholder implementation)
def elements_to_geojson(elements):
    # Implement according to your needs
    return {"type": "FeatureCollection", "features": []}

# Function to fetch data from Overpass API (placeholder implementation)
def fetch_overpass(query, endpoint):
    # Implement according to your needs
    return {"elements": []}

# Fixed the function name from 'ite_csv' to 'write_csv'å
def write_csv(rows: List[Dict[str, Any]], path: str) -> None:
    fieldnames = [
        "osm_type", "osm_id", "name", "addr_street", "addr_housenumber",
        "addr_postcode", "addr_city", "opening_hours", "wheelchair",
        "phone", "email", "website", "lat", "lon"
    ]
    with open(path, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for el in rows:
            row = {k: el.get(k) for k in fieldnames}
            writer.writerow(row)

def write_geojson(geo: Dict[str, Any], path: str) -> None:
    with open(path, "w", encoding="utf-8") as f:
        json.dump(geo, f, ensure_ascii=False, indent=2)

def fetch_and_save(endpoint: str, out_prefix: str, fmt: str) -> Tuple[str, str]:
    data = fetch_overpass(OVERPASS_QUERY, endpoint=endpoint)
    elements = data.get("elements", [])
    rows = [normalize_element(e) for e in elements]
    csv_path = geojson_path = None
    if fmt in ("csv", "both"):
        csv_path = f"{out_prefix}.csv"
        write_csv(rows, csv_path)
    if fmt in ("geojson", "both"):
        geojson_path = f"{out_prefix}.geojson"
        write_geojson(elements_to_geojson(elements), geojson_path)
    return csv_path, geojson_path

def main(argv=None):
    parser = argparse.ArgumentParser(description="Fetch Berlin dental offices from OSM")
    parser.add_argument("--endpoint", default=DEFAULT_ENDPOINT)
    parser.add_argument("--out", dest="out_prefix", default="berlin_dentists")
    parser.add_argument("--format", choices=["csv", "geojson", "both"], default="both")
    args = parser.parse_args(argv)
    if USER_AGENT.endswith("you@example.com"):
        print("[WARN] Please customize USER_AGENT.", file=sys.stderr)
    csv_path, geojson_path = fetch_and_save(args.endpoint, args.out_prefix, args.format)
    if csv_path:
        print(f"Saved CSV: {csv_path}")
    if geojson_path:
        print(f"Saved GeoJSON: {geojson_path}")

if __name__ == "__main__":
    import sys
    args_to_parse = sys.argv[1:]
    try:
        # Remove Jupyter's extra args if present
        if any(arg.endswith(".json") for arg in args_to_parse):
            args_to_parse = []
        main_parser = argparse.ArgumentParser(...)
        # define arguments...
        parsed_args = main_parser.parse_args(args_to_parse)
    except SystemExit:
        pass  # Prevent Jupyter from exiting

In [2]:
#!/usr/bin/env python3
"""
Retrieve dental offices/clinics in Berlin from OpenStreetMap via Overpass API.

✅ Jupyter-friendly (ignores unknown CLI args)
✅ Adds clear diagnostics for HTTP 400 responses (shows Overpass error text)
✅ Tries multiple mirrors automatically ("--endpoint auto")
✅ Falls back between two safe queries (by area-id and by name) to avoid syntax quirks
✅ Optional: choose output (CSV / GeoJSON / both)
✅ Includes lightweight self-tests you can run in-notebook

USAGE (terminal or Jupyter cell):
    # defaults: auto mirror, both CSV+GeoJSON, files named berlin_dentists.*
    main()

    # custom
    main([
        "--format", "both",
        "--out", "berlin_dentists",
        "--endpoint", "auto",
        "--user-agent", "BerlinDentistsFetcher/1.0 (contact: your@email)"
    ])

Notes:
- Please **set a meaningful User-Agent** (email or URL) to be a good Overpass citizen.
- If a 400 happens, this script now prints the server's error text and automatically
  retries with a different mirror and alternate query.
"""
from __future__ import annotations

import argparse
import csv
import json
import os
import sys
import time
from typing import Any, Dict, Iterable, List, Optional, Tuple

import requests

# --------------------------- Config & Queries --------------------------- #
DEFAULT_ENDPOINT = "auto"  # "auto" tries MIRRORS in order below
MIRRORS: List[str] = [
    "https://overpass.kumi.systems/api/interpreter",
    "https://overpass-api.de/api/interpreter",
    "https://z.overpass-api.de/api/interpreter",
]

# Berlin relation id is 62422; Overpass area id is 3600000000 + 62422
BERLIN_REL_ID = 62422
BERLIN_AREA_ID = 3600000000 + BERLIN_REL_ID  # 3600062422

# Primary query: use area by numeric id (more robust, faster)
OVERPASS_QUERY_BY_ID = rf"""
[out:json][timeout:180];
area({BERLIN_AREA_ID})->.searchArea;
(
  node["amenity"="dentist"](area.searchArea);
  way["amenity"="dentist"](area.searchArea);
  relation["amenity"="dentist"](area.searchArea);
);
out tags center;
"""

# Fallback query: use name/admin filter (in case a mirror has trouble with numeric area ids)
OVERPASS_QUERY_BY_NAME = r"""
[out:json][timeout:180];
area["name"="Berlin"]["boundary"="administrative"]["admin_level"~"^(4|6)$"];->.searchArea;
(
  node["amenity"="dentist"](area.searchArea);
  way["amenity"="dentist"](area.searchArea);
  relation["amenity"="dentist"](area.searchArea);
);
out tags center;
"""

# Default UA can be overridden via --user-agent or OSM_USER_AGENT env var
DEFAULT_USER_AGENT = (
    os.getenv("OSM_USER_AGENT")
    or "BerlinDentistsFetcher/1.0 (contact: set-your-contact@example.com)"
)

# --------------------------- Exceptions --------------------------- #
class OverpassError(RuntimeError):
    pass

class OverpassBadRequest(OverpassError):
    def __init__(self, message: str, response_text: str = ""):
        super().__init__(message)
        self.response_text = response_text

# --------------------------- Core Fetch Logic --------------------------- #

def _headers(user_agent: str) -> Dict[str, str]:
    return {
        "User-Agent": user_agent,
        "Content-Type": "application/x-www-form-urlencoded",
        "Accept": "application/json",
    }


def fetch_overpass_single(query: str, endpoint: str, user_agent: str, timeout_s: int = 300) -> Dict[str, Any]:
    """POST a single Overpass query to one endpoint, raising detailed errors.

    Raises OverpassBadRequest on HTTP 400 with server error text included.
    """
    resp = requests.post(
        endpoint,
        data={"data": query},
        headers=_headers(user_agent),
        timeout=timeout_s,
    )
    if resp.status_code == 400:
        # Show server error to aid debugging (syntax errors, etc.)
        text = (resp.text or "").strip()
        raise OverpassBadRequest(
            f"400 Bad Request from {endpoint}", response_text=text
        )
    # Retry logic for 429/5xx can be handled by caller; here we surface details
    resp.raise_for_status()
    return resp.json()


def fetch_overpass_with_retries(
    queries: Iterable[str],
    endpoint: str,
    user_agent: str,
    max_retries: int = 6,
    base_backoff: float = 1.6,
) -> Dict[str, Any]:
    """Try queries (first to last) against endpoint with exponential backoff."""
    last_err: Optional[Exception] = None
    for attempt in range(max_retries):
        try:
            for q in queries:
                return fetch_overpass_single(q, endpoint, user_agent)
        except OverpassBadRequest as e:
            # Don't retry on 400 unless it's the *first* query and we can try the fallback query
            last_err = e
            # If first query failed (likely BY_ID) and we have a second (BY_NAME), try it next attempt
            if attempt == 0:
                # sleep a tiny bit then continue loop to try next query
                time.sleep(0.2)
                continue
            break
        except (requests.ConnectionError, requests.Timeout, requests.HTTPError) as e:
            last_err = e
            # 429 or 5xx: backoff and retry
            sleep_s = min(90.0, base_backoff ** (attempt + 1))
            time.sleep(sleep_s)
            continue
    # Exhausted retries
    if isinstance(last_err, OverpassBadRequest):
        msg = f"Overpass 400 error. Server message (truncated):\n{last_err.response_text[:500]}"
        raise OverpassBadRequest(msg, response_text=getattr(last_err, "response_text", ""))
    raise OverpassError(f"Failed after {max_retries} attempts: {last_err}")


def fetch_overpass_auto(
    query_primary: str = OVERPASS_QUERY_BY_ID,
    query_fallback: str = OVERPASS_QUERY_BY_NAME,
    endpoint: str = DEFAULT_ENDPOINT,
    user_agent: str = DEFAULT_USER_AGENT,
) -> Dict[str, Any]:
    """Try multiple mirrors and query fallbacks. Returns parsed JSON."""
    queries = (query_primary, query_fallback)

    endpoints: List[str]
    if endpoint == "auto":
        endpoints = MIRRORS
    else:
        endpoints = [endpoint]

    last_err: Optional[Exception] = None
    for url in endpoints:
        try:
            return fetch_overpass_with_retries(queries, url, user_agent)
        except Exception as e:  # capture details and move to next mirror
            last_err = e
            continue
    raise OverpassError(f"All endpoints failed. Last error: {last_err}")

# --------------------------- Transform --------------------------- #

def normalize_element(el: Dict[str, Any]) -> Dict[str, Any]:
    el_type = el.get("type")
    osm_id = el.get("id")
    tags = el.get("tags", {}) or {}

    # Coordinates: for nodes use lat/lon; for ways/relations use 'center'
    if el_type == "node":
        lat = el.get("lat")
        lon = el.get("lon")
    else:
        center = el.get("center", {})
        lat = center.get("lat")
        lon = center.get("lon")

    phone = tags.get("phone") or tags.get("contact:phone")
    email = tags.get("email") or tags.get("contact:email")
    website = tags.get("website") or tags.get("contact:website")

    return {
        "osm_type": el_type,
        "osm_id": osm_id,
        "name": tags.get("name"),
        "addr_street": tags.get("addr:street"),
        "addr_housenumber": tags.get("addr:housenumber"),
        "addr_postcode": tags.get("addr:postcode"),
        "addr_city": tags.get("addr:city"),
        "opening_hours": tags.get("opening_hours"),
        "wheelchair": tags.get("wheelchair"),
        "phone": phone,
        "email": email,
        "website": website,
        "lat": lat,
        "lon": lon,
        "_all_tags": tags,
    }


def elements_to_geojson(elements: List[Dict[str, Any]]) -> Dict[str, Any]:
    features = []
    for el in elements:
        norm = normalize_element(el)
        lat, lon = norm.get("lat"), norm.get("lon")
        if lat is None or lon is None:
            continue
        props = {k: v for k, v in norm.items() if k not in {"lat", "lon"}}
        features.append(
            {
                "type": "Feature",
                "geometry": {"type": "Point", "coordinates": [lon, lat]},
                "properties": props,
            }
        )
    return {"type": "FeatureCollection", "features": features}


# --------------------------- I/O --------------------------- #

def write_csv(rows: List[Dict[str, Any]], path: str) -> None:
    fieldnames = [
        "osm_type",
        "osm_id",
        "name",
        "addr_street",
        "addr_housenumber",
        "addr_postcode",
        "addr_city",
        "opening_hours",
        "wheelchair",
        "phone",
        "email",
        "website",
        "lat",
        "lon",
    ]
    with open(path, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for el in rows:
            row = {k: el.get(k) for k in fieldnames}
            writer.writerow(row)


def write_geojson(geo: Dict[str, Any], path: str) -> None:
    with open(path, "w", encoding="utf-8") as f:
        json.dump(geo, f, ensure_ascii=False, indent=2)


# --------------------------- High-level API --------------------------- #

def fetch_and_save(
    endpoint: str,
    out_prefix: str,
    fmt: str,
    user_agent: str,
) -> Tuple[Optional[str], Optional[str]]:
    data = fetch_overpass_auto(
        query_primary=OVERPASS_QUERY_BY_ID,
        query_fallback=OVERPASS_QUERY_BY_NAME,
        endpoint=endpoint,
        user_agent=user_agent,
    )
    elements = data.get("elements", [])
    rows = [normalize_element(e) for e in elements]

    csv_path: Optional[str] = None
    geojson_path: Optional[str] = None

    if fmt in ("csv", "both"):
        csv_path = f"{out_prefix}.csv"
        write_csv(rows, csv_path)
    if fmt in ("geojson", "both"):
        geojson_path = f"{out_prefix}.geojson"
        write_geojson(elements_to_geojson(elements), geojson_path)

    return csv_path, geojson_path


# --------------------------- CLI / Notebook entry --------------------------- #

def main(argv: Optional[List[str]] = None):
    parser = argparse.ArgumentParser(description="Fetch Berlin dental offices from OSM (amenity=dentist)")
    parser.add_argument("--endpoint", default=DEFAULT_ENDPOINT, help='Overpass endpoint or "auto" to try multiple mirrors')
    parser.add_argument("--out", dest="out_prefix", default="berlin_dentists", help="Output file prefix without extension")
    parser.add_argument("--format", choices=["csv", "geojson", "both"], default="both", help="Output format")
    parser.add_argument("--user-agent", default=DEFAULT_USER_AGENT, help="HTTP User-Agent (please include contact)")
    parser.add_argument("--self-test", action="store_true", help="Run self-tests and exit")
    args, _ = parser.parse_known_args(argv)

    if args.self_test:
        _run_self_tests()
        return

    if "set-your-contact" in args.user_agent:
        print(
            "[WARN] Please set --user-agent with an email or URL (Overpass etiquette).",
            file=sys.stderr,
        )

    try:
        csv_path, geojson_path = fetch_and_save(args.endpoint, args.out_prefix, args.format, args.user_agent)
    except OverpassBadRequest as e:
        # Print full server message to help users fix query issues
        print("[ERROR] Overpass returned 400 Bad Request.", file=sys.stderr)
        if e.response_text:
            snippet = e.response_text.strip()
            if len(snippet) > 2000:
                snippet = snippet[:2000] + "\n… (truncated)"
            print("--- Server response begin ---", file=sys.stderr)
            print(snippet, file=sys.stderr)
            print("--- Server response end ---", file=sys.stderr)
        raise
    if csv_path:
        print(f"Saved CSV: {csv_path}")
    if geojson_path:
        print(f"Saved GeoJSON: {geojson_path}")


# --------------------------- Self-tests --------------------------- #

def _sample_overpass_elements() -> List[Dict[str, Any]]:
    # Minimal fixture with a node and a way (with center)
    return [
        {
            "type": "node",
            "id": 123,
            "lat": 52.52,
            "lon": 13.405,
            "tags": {
                "amenity": "dentist",
                "name": "Dr. Node",
                "addr:street": "Alexanderplatz",
                "addr:housenumber": "1",
                "addr:postcode": "10178",
                "addr:city": "Berlin",
                "phone": "+49 30 123456",
                "website": "https://example.com",
            },
        },
        {
            "type": "way",
            "id": 456,
            "center": {"lat": 52.5, "lon": 13.4},
            "tags": {
                "amenity": "dentist",
                "name": "Zahnzentrum Way",
                "contact:phone": "+49 30 654321",
                "contact:website": "https://way.example.com",
            },
        },
    ]


def _run_self_tests() -> None:
    elems = _sample_overpass_elements()
    rows = [normalize_element(e) for e in elems]

    # Test 1: node lat/lon retained, way center used
    assert rows[0]["lat"] == 52.52 and rows[0]["lon"] == 13.405, "Node coordinates broken"
    assert rows[1]["lat"] == 52.5 and rows[1]["lon"] == 13.4, "Way center coordinates broken"

    # Test 2: contact fields preference
    assert rows[0]["phone"] == "+49 30 123456", "Phone selection failed"
    assert rows[1]["website"] == "https://way.example.com", "Website contact selection failed"

    # Test 3: GeoJSON conversion produces two points with properties
    geo = elements_to_geojson(elems)
    assert geo["type"] == "FeatureCollection" and len(geo["features"]) == 2, "GeoJSON feature count wrong"
    for feat in geo["features"]:
        assert feat["geometry"]["type"] == "Point", "GeoJSON geometry type wrong"
        assert "properties" in feat and feat["properties"].get("osm_id") is not None, "GeoJSON properties missing"

    print("Self-tests passed ✅")


if __name__ == "__main__":
    main()


[WARN] Please set --user-agent with an email or URL (Overpass etiquette).


Saved CSV: berlin_dentists.csv
Saved GeoJSON: berlin_dentists.geojson


In [3]:
main([
    "--endpoint", "auto",
    "--format", "both",
    "--out", "berlin_dentists",
    "--user-agent", "BerlinDentistsFetcher/1.0 (contact: jaywindie@gmail.com)"
])

Saved CSV: berlin_dentists.csv
Saved GeoJSON: berlin_dentists.geojson


In [4]:
import pandas as pd
import os

# Load the CSV
input_file = "/Users/jamie/berlin_dentists.csv"  
df = pd.read_csv(input_file)

# -----------------------------
# Step 1: Inspect Data
# -----------------------------
print("Initial Data Info:")
print(df.info())
print(df.head())

# -----------------------------
# Step 2: Normalize Text Fields
# -----------------------------
df['name'] = df['name'].fillna('Unknown').str.strip().str.title()
df['addr_street'] = df['addr_street'].fillna('').str.strip().str.title()
df['addr_housenumber'] = df['addr_housenumber'].fillna('').astype(str)
df['addr_city'] = df['addr_city'].fillna('Berlin').str.strip().str.title()
df['addr_postcode'] = df['addr_postcode'].fillna('').astype(str)

# Combine street + house number
df['address_full'] = df['addr_street'] + ' ' + df['addr_housenumber']

# -----------------------------
# Step 3: Remove Duplicates
# -----------------------------
df = df.drop_duplicates(subset=['name', 'address_full'])

# -----------------------------
# Step 4: Normalize Phone Numbers
# -----------------------------
df['phone'] = df['phone'].fillna('').str.replace(' ', '').str.replace('-', '')

# -----------------------------
# Step 5: Standardize Wheelchair Column
# -----------------------------
df['wheelchair'] = df['wheelchair'].fillna('unknown').str.lower()

# -----------------------------
# Step 6: Ensure Coordinates Are Correct
# -----------------------------
for col in ['lat', 'lon']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
df = df.dropna(subset=['lat', 'lon'])

# -----------------------------
# Step 7: Save Cleaned CSV
# -----------------------------
output_file = os.path.join(os.path.dirname(input_file), "berlin_dentists_clean.csv")
df.to_csv(output_file, index=False)
print(f"Transformed data saved to {output_file}")

Initial Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   osm_type          780 non-null    object 
 1   osm_id            780 non-null    int64  
 2   name              754 non-null    object 
 3   addr_street       568 non-null    object 
 4   addr_housenumber  568 non-null    object 
 5   addr_postcode     528 non-null    float64
 6   addr_city         520 non-null    object 
 7   opening_hours     586 non-null    object 
 8   wheelchair        287 non-null    object 
 9   phone             452 non-null    object 
 10  email             129 non-null    object 
 11  website           413 non-null    object 
 12  lat               780 non-null    float64
 13  lon               780 non-null    float64
dtypes: float64(3), int64(1), object(10)
memory usage: 85.4+ KB
None
  osm_type     osm_id                 name     addr_str

In [5]:
pd.read_csv("berlin_dentists_clean.csv").head()

,osm_type,osm_id,name,addr_street,addr_housenumber,addr_postcode,addr_city,opening_hours,wheelchair,phone,email,website,lat,lon,address_full
0,node,304183504,Unknown,Hönower Straße,75,12623.0,Berlin,NaN,unknown,NaN,NaN,NaN,52.511411,13.612096,Hönower Straße 75
1,node,313539258,Zahnzentrum Wedding,Müllerstraße,34a,13353.0,Berlin,Mo 09:00-19:00; Tu 09:00-18:00; We 09:00-17:00...,yes,NaN,NaN,NaN,52.548838,13.355305,Müllerstraße 34a
2,node,325161442,A. Nejad,NaN,NaN,NaN,Berlin,Mo-Tu 09:00-19:00; We 09:00-14:00; Th 09:00-19...,yes,+49303619106,NaN,NaN,52.508843,13.180477,
3,node,345236220,Dr. Beate Lengert,Kurfürstendamm,218,10719.0,Berlin,NaN,unknown,NaN,NaN,http://www.dr-beate-lengert.de/,52.502722,13.328137,Kurfürstendamm 218
4,node,391394177,Serpil Hartfiel,Kollwitzstraße,77,10435.0,Berlin,Mo-Th 08:00-19:00; Fr 08:00-12:00,no,NaN,NaN,https://www.zahnarztpraxis-hartfiel.de/,52.537547,13.418994,Kollwitzstraße 77


In [8]:
!pip install geopy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 2.3 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.8 MB/s eta 0:00:00


In [11]:
!pip install geopandas shapely pyproj fiona rtree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.4/338.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 33.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 10.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 28.8 MB/s eta 0:00:00a 0:00:01


In [19]:
cache = {}

def get_bezirk(lat, lon):
    key = (round(lat, 5), round(lon, 5))  # rounded to avoid duplicates
    if key in cache:
        return cache[key]
    try:
        location = geolocator.reverse((lat, lon), exactly_one=True, language='de')
        sleep(1)
        if location and "address" in location.raw:
            address = location.raw["address"]
            bezirk = (
                address.get("city_district") or
                address.get("borough") or
                address.get("county")
            )
            cache[key] = bezirk
            return bezirk
    except Exception as e:
        return None

In [28]:
import requests

url = "https://tsb-opendata.s3.eu-central-1.amazonaws.com/bezirksgrenzen/bezirksgrenzen.geojson"
resp = requests.get(url)
with open("berlin_bezirksgeo.json", "wb") as f:
    f.write(resp.content)
print("GeoJSON downloaded as berlin_bezirksgeo.json")

GeoJSON downloaded as berlin_bezirksgeo.json


In [36]:
!pip install geopandas requests

import geopandas as gpd
import pandas as pd
import os
import requests

# Download district GeoJSON if it's not there
geojson_path = "berlin_bezirksgeo.json"
if not os.path.exists(geojson_path):
    url = "https://tsb-opendata.s3.eu-central-1.amazonaws.com/bezirksgrenzen/bezirksgrenzen.geojson"
    print("Downloading district boundaries...")
    resp = requests.get(url)
    resp.raise_for_status()
    with open(geojson_path, "wb") as f:
        f.write(resp.content)
    print("Downloaded:", geojson_path)

# Load the GeoJSON
districts = gpd.read_file(geojson_path)

# Inspect the columns (to adjust names if needed)
print("Available columns in district GeoJSON:", districts.columns.tolist())

# Load your existing U-Bahn dataset
df = pd.read_csv("ubahn_with_neighborhoods.csv")

# Print column names to verify what's available
print("Available columns in U-Bahn dataset:", df.columns.tolist())

# Convert to GeoDataFrame - use the correct column names from your dataset
# Assuming your coordinates columns might be named differently, like 'lon'/'lat' or 'x'/'y'
# Replace 'lon' and 'lat' with your actual column names
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["lon"], df["lat"]),  # Changed from longitude/latitude to lon/lat
    crs="EPSG:4326"
)

# Ensure same CRS
districts = districts.to_crs("EPSG:4326")

# Spatial join: assign each station to a district
joined = gpd.sjoin(gdf, districts, how="left", predicate="within")

# Rename relevant columns (adjust if GeoJSON names differ)
joined = joined.rename(columns={
    "BEZIRK": "district",
    "BEZIRKNR": "district_id"
})

# Drop geometry
result = joined.drop(columns="geometry")

# Save final file
result.to_csv("ubahn_with_districts.csv", index=False, encoding="utf-8")
print("✅ File saved as ubahn_with_districts.csv")

Available columns in district GeoJSON: ['gml_id', 'Gemeinde_name', 'Gemeinde_schluessel', 'Land_name', 'Land_schluessel', 'Schluessel_gesamt', 'geometry']
Available columns in U-Bahn dataset: ['osm_type', 'osm_id', 'name', 'addr_street', 'addr_housenumber', 'addr_postcode', 'addr_city', 'opening_hours', 'wheelchair', 'phone', 'email', 'website', 'lat', 'lon', 'address_full']
✅ File saved as ubahn_with_districts.csv


In [31]:
pd.read_csv("ubahn_with_districts.csv").head()

,osm_type,osm_id,name,addr_street,addr_housenumber,addr_postcode,addr_city,opening_hours,wheelchair,phone,...,lat,lon,address_full,index_right,gml_id,Gemeinde_name,Gemeinde_schluessel,Land_name,Land_schluessel,Schluessel_gesamt
0,node,304183504,Unknown,Hönower Straße,75,12623.0,Berlin,NaN,unknown,NaN,...,52.511411,13.612096,Hönower Straße 75,6,s_wfs_alkis_bezirk.F176__7,Marzahn-Hellersdorf,10,Berlin,11,11000010
1,node,313539258,Zahnzentrum Wedding,Müllerstraße,34a,13353.0,Berlin,Mo 09:00-19:00; Tu 09:00-18:00; We 09:00-17:00...,yes,NaN,...,52.548838,13.355305,Müllerstraße 34a,9,s_wfs_alkis_bezirk.F176__10,Mitte,1,Berlin,11,11000001
2,node,325161442,A. Nejad,NaN,NaN,NaN,Berlin,Mo-Tu 09:00-19:00; We 09:00-14:00; Th 09:00-19...,yes,+49303619106,...,52.508843,13.180477,,7,s_wfs_alkis_bezirk.F176__8,Spandau,5,Berlin,11,11000005
3,node,345236220,Dr. Beate Lengert,Kurfürstendamm,218,10719.0,Berlin,NaN,unknown,NaN,...,52.502722,13.328137,Kurfürstendamm 218,1,s_wfs_alkis_bezirk.F176__2,Charlottenburg-Wilmersdorf,4,Berlin,11,11000004
4,node,391394177,Serpil Hartfiel,Kollwitzstraße,77,10435.0,Berlin,Mo-Th 08:00-19:00; Fr 08:00-12:00,no,NaN,...,52.537547,13.418994,Kollwitzstraße 77,3,s_wfs_alkis_bezirk.F176__4,Pankow,3,Berlin,11,11000003


In [59]:
import geopandas as gpd
import pandas as pd

# ------------------------------------------------------------------
# 1. Load Bezirke polygons (already downloaded as berlin_bezirksgeo.json)
# ------------------------------------------------------------------
districts = gpd.read_file("berlin_bezirksgeo.json")
print("District file columns:", districts.columns.tolist())

# Rename German → English
districts = districts.rename(columns={
    "gml_id": "district_id",
    "Gemeinde_name": "district",
    "Gemeinde_schluessel": "district_code",
    "Land_name": "state_name",
    "Land_schluessel": "state_code",
    "Schluessel_gesamt": "full_offical_code"
})

# ------------------------------------------------------------------
# 2. Load your existing U-Bahn dataset (dentists dataset)
# ------------------------------------------------------------------
df = pd.read_csv("ubahn_with_districts.csv")
print("U-Bahn dataset columns:", df.columns.tolist())

# ------------------------------------------------------------------
# 3. Convert to GeoDataFrame using lat/lon
# ------------------------------------------------------------------
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["lon"], df["lat"]),
    crs="EPSG:4326"
)

# Ensure CRS matches
districts = districts.to_crs("EPSG:4326")

# ------------------------------------------------------------------
# 4. Spatial join: assign each station/dentist to a district
# ------------------------------------------------------------------
joined = gpd.sjoin(gdf, districts, how="left", predicate="within")

# ------------------------------------------------------------------
# 5. Keep original dataset + enriched district info
# ------------------------------------------------------------------
keep_cols = list(df.columns) + [
    "district", "district_id", "district_code",
    "state_name", "state_code", "full_offical_code"
]
result = joined[keep_cols]

# ------------------------------------------------------------------
# 6. Save enriched dataset
# ------------------------------------------------------------------
result.to_csv("ubahn_with_districts.csv", index=False, encoding="utf-8")
print("✅ File saved as ubahn_with_districts.csv with district + codes")

District file columns: ['gml_id', 'Gemeinde_name', 'Gemeinde_schluessel', 'Land_name', 'Land_schluessel', 'Schluessel_gesamt', 'geometry']
U-Bahn dataset columns: ['osm_type', 'osm_id', 'name', 'addr_street', 'addr_housenumber', 'addr_postcode', 'addr_city', 'opening_hours', 'wheelchair', 'phone', 'email', 'website', 'lat', 'lon', 'address_full', 'index_right', 'gml_id', 'Gemeinde_name', 'Gemeinde_schluessel', 'Land_name', 'Land_schluessel', 'Schluessel_gesamt']


ValueError: 'index_right' cannot be a column name in the frames being joined

In [62]:
# Your previous code remains the same until the spatial join

# ------------------------------------------------------------------
# 4. Spatial join: assign each station/dentist to a district
# ------------------------------------------------------------------
# Use suffixes to avoid conflicts
joined = gpd.sjoin(
    gdf,
    districts,
    how="left",
    predicate="within"  # Removed suffixes to keep original column names
)

# ------------------------------------------------------------------
# 5. Keep original dataset + enriched district info
# ------------------------------------------------------------------
# Now we can use the original column names
keep_cols = list(df.columns) + [
    "district", "district_id", "district_code",
    "state_name", "state_code", "full_offical_code"
]

# Check if columns exist before selecting them
available_cols = [col for col in keep_cols if col in joined.columns]
result = joined[available_cols]

# ------------------------------------------------------------------
# 6. Save enriched dataset
# ------------------------------------------------------------------
result.to_csv("ubahn_with_districts.csv", index=False, encoding="utf-8")
print("✅ File saved as ubahn_with_districts.csv with district + codes (no German columns)")

ValueError: 'index_right' cannot be a column name in the frames being joined

In [60]:
# ------------------------------------------------------------------
# 4. Spatial join: assign each station/dentist to a district
# ------------------------------------------------------------------
# Use op_prefix parameter to avoid column name conflicts
joined = gpd.sjoin(gdf, districts, how="left", predicate="within", lsuffix="_left", rsuffix="_right")

# ------------------------------------------------------------------
# 5. Keep original dataset + enriched district info
# ------------------------------------------------------------------
keep_cols = list(df.columns) + [
    "district", "district_id", "district_code",
    "state_name", "state_code", "full_offical_code"
]
result = joined[keep_cols]

# ------------------------------------------------------------------
# 6. Save enriched dataset
# ------------------------------------------------------------------
result.to_csv("ubahn_with_districts.csv", index=False, encoding="utf-8")
print("✅ File saved as ubahn_with_districts.csv with district + codes")

✅ File saved as ubahn_with_districts.csv with district + codes


In [65]:
pd.read_csv("ubahn_with_districts.csv").head()

,osm_type,osm_id,name,addr_street,addr_housenumber,addr_postcode,addr_city,opening_hours,wheelchair,phone,...,Gemeinde_schluessel,Land_name,Land_schluessel,Schluessel_gesamt,district,district_id,district_code,state_name,state_code,full_offical_code
0,node,304183504,Unknown,Hönower Straße,75,12623.0,Berlin,NaN,unknown,NaN,...,10,Berlin,11,11000010,Marzahn-Hellersdorf,s_wfs_alkis_bezirk.F176__7,10,Berlin,11,11000010
1,node,313539258,Zahnzentrum Wedding,Müllerstraße,34a,13353.0,Berlin,Mo 09:00-19:00; Tu 09:00-18:00; We 09:00-17:00...,yes,NaN,...,1,Berlin,11,11000001,Mitte,s_wfs_alkis_bezirk.F176__10,1,Berlin,11,11000001
2,node,325161442,A. Nejad,NaN,NaN,NaN,Berlin,Mo-Tu 09:00-19:00; We 09:00-14:00; Th 09:00-19...,yes,+49303619106,...,5,Berlin,11,11000005,Spandau,s_wfs_alkis_bezirk.F176__8,5,Berlin,11,11000005
3,node,345236220,Dr. Beate Lengert,Kurfürstendamm,218,10719.0,Berlin,NaN,unknown,NaN,...,4,Berlin,11,11000004,Charlottenburg-Wilmersdorf,s_wfs_alkis_bezirk.F176__2,4,Berlin,11,11000004
4,node,391394177,Serpil Hartfiel,Kollwitzstraße,77,10435.0,Berlin,Mo-Th 08:00-19:00; Fr 08:00-12:00,no,NaN,...,3,Berlin,11,11000003,Pankow,s_wfs_alkis_bezirk.F176__4,3,Berlin,11,11000003
